# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.pipeline import FeatureUnion, Pipeline

from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

import joblib
import pickle

In [3]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')


df = pd.read_sql_table('InsertTableName', 'sqlite:///InsertDatabaseName.db')  
X = df["message"].values               #Feature Matrix
y = df[df.columns[4:]].values           #Target Variable

In [22]:
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [23]:
df

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,14,Information about the National Palace-,Informtion au nivaux palais nationl,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,15,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,16,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
8,17,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,18,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1


### 2. Write a tokenization function to process your text data

In [4]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

In [5]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

In [6]:
#[tokenize(x) for x in X].size        

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
        ])



### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [28]:
model = pipeline
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [31]:
y_test

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 1, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [33]:
for i in range(0,len(y_pred.T)-1):
    print(classification_report(y_test.T[i], y_pred.T[i]))


             precision    recall  f1-score   support

          0       0.63      0.46      0.53      1542
          1       0.85      0.91      0.88      5010
          2       0.21      0.33      0.26        45

avg / total       0.79      0.80      0.79      6597

             precision    recall  f1-score   support

          0       0.90      0.97      0.93      5481
          1       0.78      0.45      0.57      1116

avg / total       0.88      0.89      0.87      6597

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6566
          1       1.00      0.10      0.18        31

avg / total       1.00      1.00      0.99      6597

             precision    recall  f1-score   support

          0       0.75      0.84      0.79      3854
          1       0.73      0.61      0.66      2743

avg / total       0.74      0.74      0.74      6597

             precision    recall  f1-score   support

          0       0.93      0.99 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [8]:

parameters = {
        'clf__estimator__n_estimators': [20, 50]
        }

cv = GridSearchCV(estimator=pipeline, param_grid=parameters, cv=3,n_jobs=-1, verbose=3)


In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y)


cv.fit(X_train, y_train)
#y_pred = model.predict(X_test)
#print("Best Parameters:", cv.best_params_)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] clf__estimator__n_estimators=20 .................................
[CV]  clf__estimator__n_estimators=20, score=0.2602698196149765, total= 1.4min
[CV] clf__estimator__n_estimators=20 .................................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.6min remaining:    0.0s


[CV]  clf__estimator__n_estimators=20, score=0.24863553668890237, total= 1.4min
[CV] clf__estimator__n_estimators=20 .................................


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  3.2min remaining:    0.0s


[CV]  clf__estimator__n_estimators=20, score=0.24393571861734384, total= 1.4min
[CV] clf__estimator__n_estimators=50 .................................
[CV]  clf__estimator__n_estimators=50, score=0.25632863422767926, total= 3.3min
[CV] clf__estimator__n_estimators=50 .................................
[CV]  clf__estimator__n_estimators=50, score=0.25758035172832017, total= 3.3min
[CV] clf__estimator__n_estimators=50 .................................
[CV]  clf__estimator__n_estimators=50, score=0.2506064281382656, total= 3.4min


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed: 16.1min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [20, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [44]:
y_pred = cv.predict(X_test)
#print(classification_report(y_pred, y_test))
#print raw accuracy score 
#print('Accuracy Score: {}'.format(np.mean(y_test == y_pred)))
for i in range(0,len(y_pred.T)-1):
    print(classification_report(y_test.T[i], y_pred.T[i]))

             precision    recall  f1-score   support

          0       0.70      0.44      0.54      1526
          1       0.85      0.94      0.89      5018
          2       0.33      0.42      0.37        53

avg / total       0.81      0.82      0.81      6597

             precision    recall  f1-score   support

          0       0.90      0.98      0.94      5491
          1       0.83      0.48      0.61      1106

avg / total       0.89      0.90      0.88      6597

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6573
          1       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      6597

             precision    recall  f1-score   support

          0       0.79      0.83      0.81      3884
          1       0.74      0.69      0.72      2713

avg / total       0.77      0.77      0.77      6597

             precision    recall  f1-score   support

          0       0.92      1.00 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:

parameters = {
        'clf__estimator__n_estimators': [20, 50]
        }

cv = GridSearchCV(estimator=pipeline, param_grid=parameters, cv=3,n_jobs=-1, verbose=3)

### 9. Export your model as a pickle file

In [45]:
with open('model.pkl', 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.